In [57]:
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from tkinter.filedialog import askopenfilename
from PIL import Image
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import numpy as np
import keras
import imutils
import os.path
import importlib
from os import listdir
from keras import backend as k
from keras.models import load_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from playsound import playsound
import IPython
from gtts import gTTS
import pygame
import time


pip install playsound

In [58]:
y_train = []
print ("I entered train")
# to get the name of the folder
for name_folder in os.listdir("C:\\Users\HP\Desktop\optical-character-recognition-OCR-master\Fnt") :
    name = 'C:\\Users\HP\Desktop\optical-character-recognition-OCR-master\Fnt/' + name_folder
    for f in listdir(name):
        # name of the folder is the name of the output
        y_train.append(np.asarray(name_folder))
y_train = np.asarray(y_train)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y_train)

I entered train


In [59]:

# Saving the converted audio in a mp3 file named 
def image_segmentation(image_name):
    #print( "I entered letter segmentation")
    # reading the image
    image = cv2.imread(image_name)

    # converting the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # threshold to convert the image to pure black and white
    thresh = cv2.threshold(gray, 0,255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]


    # find the contours (continous blob of pixels ) in the image 
    contours = cv2.findContours(thresh,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Hack for compatibility with different OpenCV versions
    contours = contours[0] 
    letter_image_regions = []

    # now loop through each of the letter in the image 
    for contour in contours:
        # get the rectangle that contains the contour
        x,y,w,h = cv2.boundingRect(contour)
        # compare the width and height of the contour to detect if it
        # has one letter or not
        if w/h >1.25:
            # this is too wide for a single letter
            continue
        elif w<3 or h<3:
            # this is a very small image probably a noise
            continue
        else:
        # this is a normal letter by itself
            letter_image_regions.append((x,y,w,h))

    return letter_image_regions



In [60]:
print("loading the trained model")
model = load_model('C:\\Users\HP\Desktop\optical-character-recognition-OCR-master\models/model5.h5')
print("loaded")
#model.summary()

loading the trained model
loaded


In [61]:
def image_for_detection(raw_image):

	#remove tiny noises by blurring
	sm_image = cv2.GaussianBlur(raw_image,(5,5),0)
	
	#binarize
	ret, bw_image = cv2.threshold(sm_image,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
	
	#dilate
	kernel = np.ones((2,2),np.uint8)
	bw_image = cv2.dilate(bw_image,kernel)
	
	return bw_image
	
def image_for_extraction(raw_image):
	
	raw_image = cv2.GaussianBlur(raw_image,(3,3),0)
	
	ret,no_sm_bw_image = cv2.threshold(raw_image,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
	
	return no_sm_bw_image
	
def getTransformationMatrix(img):
	#input should be a binarized image - text white, bg black
	
	#Find all white pixels
	pts = np.empty([0,0])
	pts = cv2.findNonZero(img)

	#Get rotated rect of white pixels
	rect = cv2.minAreaRect(pts)
	
	# rect[0] has the center of rectangle, rect[1] has width and height, rect[2] has the angle
	# To draw the rotated box and save the png image, uncomment below
	drawrect = img.copy()
	drawrect = cv2.cvtColor(drawrect, cv2.COLOR_GRAY2BGR)
	box = cv2.boxPoints(rect)
	box = np.int0(box) # box now has four vertices of rotated rectangle
	cv2.drawContours(drawrect,[box],0,(0,0,255),10)
	cv2.imwrite('rotated_rect.png', drawrect)

	#Change rotation angle if the tilt is in another direction
	rect = list(rect)
	if (rect[1][0] < rect[1][1]): # rect.size.width > rect.size.height
		temp = list(rect[1])
		temp[0], temp[1] = temp[1], temp[0]
		rect[1] = tuple(temp)
		rect[2] = rect[2] + 90.0

	#convert rect back to numpy/tuple
	rect = np.asarray(rect)
	
	#Rotate the image according to the found angle
	rotated_image = np.empty([0,0])
	M = cv2.getRotationMatrix2D(rect[0], rect[2], 1.0)
	#img = cv2.warpAffine(img, M, (img.shape[1],img.shape[0]))

	#returns the transformation matrix for this rotation
	return M

def rotate(image, M):
	return cv2.warpAffine(image, M, (image.shape[1],image.shape[0]))

def findLines(bw_image, LinesThres):
	# making horizontal projection
	horProj = cv2.reduce(bw_image, 100, cv2.REDUCE_AVG)

	# make hist - same dimension as horProj - if 0 (space), then True, else False
	th = 0; # black pixels threshold value. this represents the space lines
	hist = horProj <= th;

	#Get mean coordinate of white white pixels groups
	ycoords = []
	y = 0
	count = 0
	isSpace = False

	for i in range(0, bw_image.shape[0]):
		
		if (not isSpace):
			if (hist[i]): #if space is detected, get the first starting y-coordinates and start count at 1
				isSpace = True
				count = 1
				y = i
		else:
			if (not hist[i]):
				isSpace = False
				#when smoothing, thin letters will breakdown, creating a new blank lines or pixel rows, but the count will be small, so we set a threshold.
				if (count >=LinesThres):
					ycoords.append(y // count)
			else:
				y = y + i
				count = count + 1

	ycoords.append(y // count)
	
	#returns y-coordinates of the lines found
	return ycoords

def LinesMedian(bw_image):
	# making horizontal projections
	
	horProj = cv2.reduce(bw_image, 1, cv2.REDUCE_AVG)

	# make hist - same dimension as horProj - if 0 (space), then True, else False
	th = 0; # black pixels threshold value. this represents the space lines
	hist = horProj <= th;

	#Get mean coordinate of white white pixels groups
	ycoords = []
	y = 0
	count = 0
	isSpace = False
	median_count = []

	for i in range(0, bw_image.shape[0]):
		
		if (not isSpace):
			if (hist[i]): #if space is detected, get the first starting y-coordinates and start count at 1
				isSpace = True
				count = 1
				#y = i
		else:
			if (not hist[i]):
				isSpace = False
				median_count.append(count)
			else:
				#y = y + i
				count = count + 1
	median_count.append(count)
	#ycoords.append(y / count)
	
	#returns counts of each blank rows of each of the lines found
	return median_count

def get_lines_threshold(percent, img_for_det):
	ThresPercent = percent
	LinMed = LinesMedian(img_for_det)
	LinMed = sorted(LinMed)
	LinesThres = LinMed[len(LinMed)//3]*(ThresPercent//100.0)
	LinesThres = int(LinesThres)
	return LinesThres

def findSpaces(line, thres_space):
	
	# making vertical projections
	
	verProj = cv2.reduce(line, 0, cv2.REDUCE_AVG)
	#print('v',verProj)
	# make hist - same dimension as horProj - if 0 (space), then True, else False
	th = 0 # black pixels threshold value. this represents the space lines
	hist = (verProj <= th)
	#print('hist',hist)
	#Get mean coordinate of white white pixels groups
	xcoords = []
	x = 0
	count = 0
	isSpace = False
	#print('shape',line.shape[1])
	for i in range(0, line.shape[1]):
		if (not isSpace):
			if (hist[0][i]): #if space is detected, get the first starting x-coordinates and start count at 1
				#print('hist i',hist[0][i],i)
				isSpace = True
				count = 1
				x = i
		else:
			if (not hist[0][i]):
				isSpace = False
				#when smoothing, thin letters will breakdown, creating a new blank lines or pixel columns, but the count will be small, so we set a threshold.
				#print count,"\t",
				if (count > thres_space):
					xcoords.append(x // count)
			else:
				x = x + i
				count = count + 1
	

	xcoords.append(x // count)
	
	return xcoords

def SpacesMedian(line):
	
	# making vertical projections
	
	verProj = cv2.reduce(line, 0, cv2.REDUCE_AVG)

	# make hist - same dimension as horProj - if 0 (space), then True, else False
	th = 0; # black pixels threshold value. this represents the space lines
	hist = verProj <= th;

	#Get mean coordinate of white white pixels groups
	xcoords = []
	x = 0
	count = 0
	isSpace = False
	median_count = []
	for i in range(0, line.shape[1]):
		if (not isSpace):
			if (hist[0][i]): #if space is detected, get the first starting x-coordinates and start count at 1
				isSpace = True
				count = 1
				#x = i
		else:
			if (not hist[0][i]):
				isSpace = False
				#when smoothing, thin letters will breakdown, creating a new blank lines or pixel columns, but the count will be small, so we set a threshold.
				#print count,"\t",
				
				#append each count of rows of blank gaps found
				median_count.append(count)
				
				#if (count > 15):
					#xcoords.append(x / count)
			else:
				#x = x + i
				count = count + 1
	
	median_count.append(count)
	xcoords.append(x // count)
	
	#returns x-coordinates of the spaces found in the line
	return median_count
	
def get_spaces_threshold(ycoords, img_for_det) :

	## Find Median for setting threshold
	medianList = []
	for i in range ( 0, len(ycoords)-1 ):
		line = img_for_det[range(ycoords[i],ycoords[i+1])]
		medianList.append(SpacesMedian(line))
	
	#medianList contains count of each blank columns found in all lines
	#including spaces found between each characters too
	
	#find the row among medianList[] with maximum length
	max_len = len(medianList[0])
	max_in = 0 #for index number
	for i in range (0, len(medianList)):
		if max_len < len(medianList[i]):
			max_len = len(medianList[i])
			max_in = i

	#sort the row  having the maximum no. of elements (decending order)
	mList = sorted(medianList[max_in],reverse=True)
	
	#delete elements produced from the page's margin
	mList = np.delete(mList, [0,1,2])
	
	firstItem = mList[0]
	for i in range (len(mList)-1, 0, -1):
		if mList[i] < firstItem//2:
			mList = np.delete(mList,i)

	mean = np.mean(mList)
	threshold_space = mean/2
	
	return threshold_space

def get_words(raw_image):
	
	#Returns a list/array of all the words found along with the number of words on each line.
	
	#preprocessing of the image
	
	#img_for_det used for detecting the character and lines boundaries
	img_for_det = image_for_detection(raw_image)
	
	#img_for_ext used for the actual extraction of the characters
	img_for_ext = image_for_extraction(raw_image)
	
	#get the rotated angle of the tilt
	M = getTransformationMatrix(img_for_det) # M is transformation matrix
	#rotate the iamge with M
	img_for_det = rotate(img_for_det,M)
	#rotate image that will be used for extraction too
	img_for_ext = rotate(img_for_ext,M)
	
	#for debugging purpose, we also write the images to files
	cv2.imwrite('img_for_detection.png', img_for_det)
	cv2.imwrite('img_for_extraction.png', img_for_ext)

	#get threshold to determine how much gap should be considered as the line gap
	LinesThres = get_lines_threshold(50, img_for_det)
	ycoords = findLines(img_for_det, LinesThres)
	#print(ycoords)
	# save image with lines printed ==========
	img_with_lines = img_for_ext.copy()
	for i in ycoords:
		cv2.line(img_with_lines,(0,i),(img_with_lines.shape[1],i),255,1)
	#cv2.imwrite('img_with_lines.png', img_with_lines)
	#==========

	### =========== lines detection finish - ===========================
	
	#calculate max_line_height on each line
	max_height_on_line = []
	for i in range ( 0, len(ycoords)-1 ): #iterate line
		
		line = img_for_ext[range(ycoords[i],ycoords[i+1])]

		# to find max_line_height of each line we find contours again in this line only
      
		#cnt_len = cv2.arcLength(cnt, True)
		contour0 = cv2.findContours(line.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
		contours = [cv2.approxPolyDP(cnt,2,True) for cnt in contour0[0]]

        # === Extract Bounding Rectangles
		maxArea = 0
		rect=[]
		for ctr in contours:
			maxArea = max(maxArea,cv2.contourArea(ctr))

		areaRatio = 0.008

		for ctr in contours:
			if cv2.contourArea(ctr) > maxArea * areaRatio: 
				rect.append(cv2.boundingRect(cv2.approxPolyDP(ctr,1,True)))
		
		#Find max_line_height and width
		max_line_height = 0
		
		#for x in rect:
		#		round(x)
		#rect.append(x)
		for i in rect:
			x = i[0]
			y = i[1]
			w = i[2]
			h = i[3]
			
			if(h>max_line_height):
				max_line_height = h
		
		max_height_on_line.append(max_line_height)
	
	### =========== space in a line detection begins ===================

	#get the threshold to determine how much gap should be considered as the space between the words
	threshold_space = get_spaces_threshold(ycoords, img_for_det)

	#split lines based on the ycoords of the detected lines
	#each line is put into the var 'line' and the words are found
	#based on the threshold_space value.

	words_on_line=[]
	all_words=[]
	count = 0
	number_of_words = 0

	for i in range ( 0, len(ycoords)-1 ): #iterate line

		line = img_for_det[range(ycoords[i],ycoords[i+1])]
		#cv2.imwrite(os.path.join("C:\\Users\\HP\\Desktop\\optical-character-recognition-OCR-master\\out\\",'line_no'+str(i)+'.png', line))
		
		#finding the x-coordinates of the spaces
		xcoords = findSpaces(line, threshold_space)
		
		#print((xcoords))
		for x in xcoords:
			cv2.line(line, (x,0), (x,line.shape[0]), 255, 1)
		#cv2.imwrite('img/'+str(i)+'.png', line)
		cv2.imwrite(os.path.join("C:\\Users\\HP\\Desktop\\optical-character-recognition-OCR-master\\out1\\" , str(i)+ ".png"),~line)
		count = 0
		
		for j in range (0, len(xcoords)-1 ): #iterate words
			
			#use image with no smoothing
			line = img_for_ext[range(ycoords[i],ycoords[i+1])]
			#print('line',line)
			word = line[:, xcoords[j]: xcoords[j+1]]
			#print(word)
			all_words.append(word)
			#cv2.imwrite('img/words/'+str(number_of_words)+'.png', word)
			cv2.imwrite(os.path.join("C:\\Users\\HP\\Desktop\\optical-character-recognition-OCR-master\\out2\\" , str(number_of_words)+ ".png"),~word)
			count = count + 1
			number_of_words = number_of_words + 1
			#Generate space here
		
		words_on_line.append(count)
		# Line Change
	
	return all_words, words_on_line, max_height_on_line




def image_segmentation1(name1):
    raw_image = cv2.imread(name1,0)
    count=0
        #get all the words (as an numpy image array), words on each line, and maximum height on that line
    all_words, words_on_line, max_height_on_line = get_words(raw_image)
    #for i in range(0, len(words_on_line)):
    #    for j in range(0, words_on_line[i]):
    #        all_characters = get_characters(all_words[count],max_height_on_line[i],i,j)
    #        cv2.imshow("all_words[count]",all_words[count])
    #        count = count + 1
    print ("no. of lines = ",len(words_on_line))
    print (words_on_line)
    print ("no. of words = ",len(all_words))


In [62]:
import re
from collections import Counter
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big_merged.txt').read()))

def P(word, N=sum(WORDS.values())): 
    #"Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    #"Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    #"Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    #"The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    #"All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)
    
def edits2(word): 
    #"All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


In [63]:
# a function to resize the image into appropriate dimensions
def resize(img):
    img = cv2.resize(img,(20,20))
    return img

In [64]:
def Openvoice():
    import os 

    # The text that you want to convert to audio 
    #file_name='/home/anu/PROJECT/OCR-MODEL-master/extract/letter/'+ text_file
    #with open(file_name,'r') as myfile:
    #    data=myfile.read().replace('\n','')
    str1 = ' '.join(words)
    mytext=str1

    print(mytext)
    # Language in which you want to convert 
    language = 'en'
 
    myobj = gTTS(text=mytext, lang=language, slow=False) 
    
    import time
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    myobj.save("C:\\Users\\HP\\Desktop\\optical-character-recognition-OCR-master\\out\\"+ timestr+'.mp3') 

    #print timestr
    #pygame.mixer.init()
    #pygame.mixer.music.load('/home/anu/PROJECT/OCR-MODEL-master/extract/output/' + timestr +'.mp3')
    #pygame.mixer.music.play()

    # Saving the converted audio in a mp3 file named 
    # welcome 
    # Playing the converted file 
    #os.system("welcome.mp3")

In [65]:

def predict():    
    listd=[]
    new=[]
    global words
    global ans

    words=[]
    path='C:\\Users\HP\Desktop\optical-character-recognition-OCR-master\out2/'
    for image in listdir(path):
        listd.append(image)
    res = [int(sub.split('.')[0]) for sub in listd] 
    res.sort()

    for image_name in res:
        a=str(image_name)+'.png'
        new.append(a)
    #print(new)


    for image_name in new:
        counter = 1
        #print(image_name)
        # constructing the name of the file 
        file_name = 'C:\\Users\HP\Desktop\optical-character-recognition-OCR-master\out2/' + image_name

        # getting segmented images 
        letters_in_image = image_segmentation(file_name)

        # sorting the letters so that letters that appear before is addressed first 
        letters_in_image = sorted(letters_in_image, key=lambda x: x[0])

        ans = ""
        ans1=""
        for (x,y,w,h) in letters_in_image:
            image = cv2.imread(file_name,0)
            letter = image[y - 2:y + h + 2, x - 2:x + w + 2]

            #cv2.imwrite(str(counter)+'.jpg', letter)
            counter = counter + 1

            letter  = resize(letter)/255
            X_test = np.asarray(letter)
            X_test = np.reshape(X_test, [-1,20,20,1])
            output = [np.argmax(model.predict(X_test, verbose = 0))]
            output = label_encoder.inverse_transform(output)
            ans1+=output[0].lower()
            ans=correction(ans1)
        words.append(ans)
        PathTextBox2.insert(END, " " + ans)
    #print(" ", ans)
    print(words)
    
    
    path='C:\\Users\\HP\\Desktop\\optical-character-recognition-OCR-master\\text\\'
    for i in words:
        global text_file
        import time
        #timestr1 = time.strftime("%Y%m%d-%H%M%S")
        text_file = open(path+timestr+'.txt', "a")
        text_file.write(i)
        text_file.write(' ')
        text_file.close()
    Openvoice()


In [66]:

    
def stop():
    pygame.mixer.music.pause()
    
def play():
    pygame.mixer.init()
    pygame.mixer.music.load('C:\\Users\\HP\\Desktop\\optical-character-recognition-OCR-master\\out\\' + timestr +'.mp3')
    pygame.mixer.music.play()
        
def Image12():
    print('in image')
    img=cv2.imread(name1)
    cv2.imshow("Input Image",img)

def clear():
    PathTextBox2.delete("1.0",END)
    PathTextBox.delete("1.0",END)
    for root, dirs, files in os.walk('C:\\Users\\HP\\Desktop\\optical-character-recognition-OCR-master\\out2\\'):
        for f in files:
            os.unlink(os.path.join(root, f))
        for d in dirs:
            shutil.rmtree(os.path.join(root, d))

    for root, dirs, files in os.walk('C:\\Users\\HP\\Desktop\\optical-character-recognition-OCR-master\\out1\\'):
        for f in files:
            os.unlink(os.path.join(root, f))
        for d in dirs:
            shutil.rmtree(os.path.join(root, d))
    import time
    global timestr
    timestr = time.strftime("%Y%m%d-%H%M%S")
    print(timestr, 'clear 1')
    
def OpenTxt():
    filename='C:\\Users\\HP\\Desktop\\optical-character-recognition-OCR-master\\text\\'+timestr+'.txt'

    #from subprocess import call
    #call("/home/anu/PROJECT/OCR-MODEL-master/extract/letter/20200427-201115.txt")
    try:
        os.system( '{0} "{1}"'.format( os.environ['EDITOR'], filename ))
    except:
         os.system( 'notepad "{0}"'.format(filename) )
    

def OpenTxt_B():
    filename='C:\\Users\\HP\\Desktop\\optical-character-recognition-OCR-master\\text2\\'+timestr+'.txt'

    #from subprocess import call
    #call("/home/anu/PROJECT/OCR-MODEL-master/extract/letter/20200427-201115.txt")
    try:
        os.system( '{0} "{1}"'.format( os.environ['EDITOR'], filename ))
    except:
         os.system( 'notepad "{0}"'.format(filename) )    
    
def translate():
    import re
    trans=[]
    braille = '⠁⠃⠉⠙⠑⠋⠛⠓⠊⠚⠅⠇⠍⠝⠕⠏⠟⠗⠎⠞⠥⠧⠺⠭⠽⠵' '⠁⠃⠉⠙⠑⠋⠛⠓⠊⠚'
    normal  = 'abcdefghijklmnopqrstuvwxyz' '1234567890'
    table = str.maketrans(normal, braille) # define the translation table

    def make_braille(data):
        data = re.sub(r'(\d+)', r'⠼\1', data) # add ⠼ before every number
        data = data.translate(table) # convert the remaining characters to braille
        return data
    print("In translate")
    
    for i in words:
        temp=make_braille(i)
        
        trans.append(temp)
        print(temp)
    path='C:\\Users\\HP\\Desktop\\optical-character-recognition-OCR-master\\text2\\'
    for i in trans:
        global text_file
        import time
        #timestr1 = time.strftime("%Y%m%d-%H%M%S")
        text_file = open(path+timestr+'.txt', "a", encoding="utf-8")
        text_file.write(i)
        text_file.write(' ')
        text_file.close() 
        
        
        
        
    
def OpenFile():
    global letter_images
    global name1
    name1 = askopenfilename(initialdir="C:\\Users\\HP\\Desktop\\optical-character-recognition-OCR-master\\images\\",
                           filetypes =(("PNG File", "*.png"),("BMP File", "*.bmp"),("JPG File", "*.jpg"),("JPEG File", "*.jpeg")),
                           title = "Choose a file."
                           ) 
    image_segmentation1(name1)
    PathTextBox.delete("1.0",END)
    PathTextBox.insert(END,name1)
   



In [ ]:
from PIL import ImageTk,Image
from tkinter import ttk
from ttkthemes import ThemedStyle
import time
global timestr
root = Tk(  )
root.resizable(0,0)      #will disable max/min tab of window
root.configure(bg="#808080")
location=str()
#root.geometry("3000X2000")
timestr = time.strftime("%Y%m%d-%H%M%S")
print(timestr,'gui time')
im = Image.open('ocrm.png')
ph = ImageTk.PhotoImage(im)
#plt.imshow(ph)

Title = root.title( "Optical Character Recognition Model")
path = StringVar()

#s = ttk.Style()
#s.configure('tab1.TFrame', background='#BB8FCE', foreground='blue')
about= " OCR model is conversion of images of typed or printed text into machine-encoded text, \n whether from a scanned document, photo of a document, a scene-photo (for example the \ntext on signs and billboards in a landscape photo). OCR is a field of research in pattern \n recognition, artificial intelligence and computer vision. The scanned page of a physical\n document can be displayed on the screen and can be read, but for the computer, it is \njust a series of black and white dots, which it cannot recognise. To enable the computer\n to read a scanned document and produce a soft copy, OCR was developed. It examines\ntext of a scanned document and translates the characters into code that makes the text \nmachine-readable so that it can be transformed into an electronic format or soft copy just\n like a document created with a word processor which users can edit, format, search\n and read.Thus, it helps computer recognize words and characters on a scanned page\n or digital images of physical printed or handwritten documents by using the optical \nproperties of words and characters printed on a scanned page or document."
contact= "A. P. SHAH INSTITUTE OF TECHNOLOGY\n\n\t PROJECT MADE BY:\n\n\t ANUJA VELASKAR\n\n\t NIDHI MUNAVALLI\n\n            APURVA WAINGANKAR"
#ADING TABSThe scanned page of a physical document can be displayed on the screen and can be read, but for the computer, it is just a series of black and white dots, which it cannot recognise. To enable the computer to read a scanned document and produce a soft copy, OCR was developed. OCR examines the text of a scanned document and translates the characters into code that makes the text machine-readable so that it can be transformed into an electronic format or soft copy just like a document created with a word processor which users can edit, format, search and read.


#ADING TABS
TAB_CONTROL = ttk.Notebook(root)


style = ttk.Style()
style = ThemedStyle(TAB_CONTROL)
style.set_theme("radiance")

style.configure("TNotebook.Tab", background="#6B6A6A", foreground="#6C7907", borderwidth=10)
style.configure('tab1.TFrame',expand="full", foreground='blue')

style.configure('TButton', background="#8EA002", foreground="black")
style.configure('TLabel', foreground="black")

TAB1 = ttk.Frame(TAB_CONTROL, style = 'tab1.TFrame')
TAB_CONTROL.add(TAB1, text= 'HOME')

TAB2 = ttk.Frame(TAB_CONTROL, style = 'tab1.TFrame')
TAB_CONTROL.add(TAB2, text='INPUT WINDOW')

TAB3 = ttk.Frame(TAB_CONTROL, style = 'tab1.TFrame')
TAB_CONTROL.add(TAB3, text='OUTPUT WINDOW')

TAB4 = ttk.Frame(TAB_CONTROL, style = 'tab1.TFrame')
TAB_CONTROL.add(TAB4, text='ABOUT OCR MODEL')

TAB5 = ttk.Frame(TAB_CONTROL, style = 'tab1.TFrame')
TAB_CONTROL.add(TAB5, text='CONTACT US')

TAB_CONTROL.pack(expand=1, fill="both")

ttk.Label(TAB4, text= about, style="TLabel", foreground="#3C4C00").pack(padx=5, pady=20)
ttk.Label(TAB5, text= contact, style="TLabel",foreground="#3C4C00").pack(padx=30, pady=50)


#tkinter.lab=Label(TAB1,image=ph).place(x=100,y=70)

#TAB_CONTROL.add(TAB1, text="profile", image=ph)#, compound=tk.TOP) # use the tk constants
try:
    lab=Label(TAB1,image=ph).place(x=60,y=45)
except TclError:
    ttk.Label(TAB5, text= home, style="TLabel").pack(anchor=CENTER,padx=30, pady=50)

#creating another frames
BTAB3 = ttk.Frame(TAB3, style = 'tab1.TFrame')
BTAB3.pack(side=BOTTOM, expand=1,pady=10)
TTAB3= ttk.Frame(TAB3, style = 'tab1.TFrame')
TTAB3.pack(side=TOP, expand=1)
CTAB3= ttk.Frame(TAB3, style = 'tab1.TFrame')
CTAB3.pack(side=TOP, expand=1)

InputLabel = ttk.Label(TAB2,text = "INPUT IMAGE",style="TLabel")
InputLabel.pack(side= TOP, pady=15)
BrowseButton = ttk.Button(TAB2,text="Browse",style="TButton",command = OpenFile)
BrowseButton.pack(side = TOP, pady=10,padx=10)

PathTextBox = Text(TAB2,width = 60,height= 2,bg="#EEF0C3", fg="black")
PathTextBox.pack(side = TOP, fill = X,padx=10)
SubmitButton = ttk.Button(TAB2,text="Submit",style="TButton",command = predict)
SubmitButton.pack(side = TOP, padx=10, pady =15) 
ClearButton = ttk.Button(TAB2,text="Start Over",style="TButton",command = clear)
ClearButton.pack(side = TOP, pady=20,padx=10)

PathLabel = ttk.Label(TTAB3,text = "Text ",style="TLabel")
PathLabel.pack(pady=7)
PathTextBox2 = Text(TTAB3,width = 60, height=3,bg="#EEF0C3", fg="black")
PathTextBox2.pack(side = TOP, fill = X,pady=10)
                
TxtLabel = ttk.Label(BTAB3,text = "Open Image                                   English Text File",style="TLabel")
TxtLabel.pack(pady=10)
ReadButton = ttk.Button(BTAB3, style="TButton",text="Image",command = Image12 )#Opens location where voice file is being stored
ReadButton.pack(padx=50, side=LEFT)
TxtButton = ttk.Button(BTAB3,text="Open",style="TButton",command = OpenTxt )#Opens location where voice file is being stored
TxtButton.pack(padx=50,side=LEFT)

PathLabel = ttk.Label(CTAB3,text = "Convert to Braille: ",style="TLabel")
PathLabel.pack(pady=10)
TButton = ttk.Button(CTAB3,text="Translate",style="TButton",command = translate )#Opens location where voice file is being stored
TButton.pack(padx=50,side=LEFT)
TxtButton = ttk.Button(CTAB3,text="Open File",style="TButton",command = OpenTxt_B )#Opens location where voice file is being stored
TxtButton.pack(padx=50,side=LEFT)

PathLabel = ttk.Label(TAB3,text = "Listen To Speech: ",style="TLabel")
PathLabel.pack(pady=10)
SpeechButton = ttk.Button(TAB3,style="TButton",text="Play",command = play )#Opens location where voice file is being stored
SpeechButton.pack(padx=125,side=LEFT)
StopButton = ttk.Button(TAB3,text="Stop",style="TButton",command = stop )#Opens location where voice file is being stored
StopButton.pack(side=LEFT)


root.mainloop()

20200918-225607 gui time
no. of lines =  3
[4, 6, 5]
no. of words =  15
['ducal', 'character', 'cognition', 'project', 'the', 'dentfcaton', 'ok', 'printed', 'characters', 'using', 'photoelectric', 'device', 'and', 'computer', 'sore']
ducal character cognition project the dentfcaton ok printed characters using photoelectric device and computer sore


conda install -c conda-forge pygobject 
